# RAG Pipline with Sentence Window Retrieval

dependencies

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.9 MB/s eta 0:00:00


In [ ]:
!pip install -qU "langchain[openai]" # select chat model OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-openai # select embeddings model OpenAI

In [ ]:
!pip install -qU langchain-community # select vector store FAISS

In [ ]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.6 MB/s eta 0:00:00


In [ ]:
!pip install llama-index

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install transformers

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI: ··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Documents

In [ ]:
file_path="/content/drive/MyDrive/ECE1508/split_documents/document_1.json"

In [ ]:
import json
from langchain_community.document_loaders import JSONLoader

def metadata_fuc(record:dict,metadata:dict)->dict:
  questions=record.get("questions",[])
  metadata["question_texts"]=[q.get("question_text") for q in questions]
  metadata["document_url"]=record.get("document_url")
  metadata["Title"] = record.get("title", "Untitled Document")
  return metadata

loader=JSONLoader(
  file_path=file_path,
  jq_schema=".",
  content_key="document_text",
  metadata_func=metadata_fuc
)

documents=loader.load()

## First Stage: Element-based Chunking

In [ ]:
from langchain_text_splitters import HTMLSectionSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]
html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_documents(documents)
html_header_splits

[Document(metadata={'source': '/content/drive/MyDrive/ECE1508/split_documents/document_1.json', 'seq_num': 1, 'question_texts': ['when did the new maze runner movie come out', 'when is the death cure coming out in ireland', 'when is the death cure released in the uk'], 'document_url': 'https://en.wikipedia.org//w/index.php?title=Maze_Runner:_The_Death_Cure&amp;oldid=828322653', 'Title': 'Untitled Document', 'Header 1': 'Untitled Document'}, page_content='Maze Runner : the Death Cure - wikipedia'),
 Document(metadata={'source': '/content/drive/MyDrive/ECE1508/split_documents/document_1.json', 'seq_num': 1, 'question_texts': ['when did the new maze runner movie come out', 'when is the death cure coming out in ireland', 'when is the death cure released in the uk'], 'document_url': 'https://en.wikipedia.org//w/index.php?title=Maze_Runner:_The_Death_Cure&amp;oldid=828322653', 'Title': 'Untitled Document', 'Header 1': 'Maze Runner : the Death Cure'}, page_content="Maze Runner : the Death Cur

In [ ]:
print(f"Split all documents into {len(html_header_splits)} sub-documents.")

Split all documents into 19 sub-documents.


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import faiss

vectorstore=FAISS.from_documents(html_header_splits,embedding=OpenAIEmbeddings())

In [ ]:
vectorstore.save_local("/content/drive/MyDrive/ECE1508/faiss_index")

In [ ]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":10})

# retrieve the top 5 contents
def retrieve_section(retriever,query):
  results=retriever.get_relevant_documents(query)
  if not results:
    return None
  top_5_match=results[:5]
  for i in range(5):
    print(f"No.{i+1} chunk: {top_5_match[i]}")

  return top_5_match

In [ ]:
queries=documents[0].metadata.get("question_texts")
print(queries)
relevant_sections=retrieve_section(retriever,queries[0])
print(relevant_sections)

['when did the new maze runner movie come out', 'when is the death cure coming out in ireland', 'when is the death cure released in the uk']
No.1 chunk: page_content='Production ( edit )     In March 2015 , it was confirmed that T.S. Nowlin , who co-wrote the first and wrote the second film , would adapt Maze Runner : The Death Cure . On September 16 , 2015 , it was confirmed that Ball would return to direct the final film .' metadata={'source': '/content/drive/MyDrive/ECE1508/split_documents/document_1.json', 'seq_num': 1, 'question_texts': ['when did the new maze runner movie come out', 'when is the death cure coming out in ireland', 'when is the death cure released in the uk'], 'document_url': 'https://en.wikipedia.org//w/index.php?title=Maze_Runner:_The_Death_Cure&amp;oldid=828322653', 'Title': 'Untitled Document', 'Header 2': 'Production ( edit )'}
No.2 chunk: page_content='Maze Runner : the Death Cure   Jump to : navigation , search For the book the film is based on , see The Dea

## Stage 2: Sentence Window Retrieval

Split chunks into smaller chunks (sentences)

In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.schema import Document as LlamaDocument

relevant_sections_llama = [
    LlamaDocument(text=doc.page_content, metadata=doc.metadata)
    for doc in relevant_sections
]
node_parser=SentenceWindowNodeParser.from_defaults(
  window_size=3,
  window_metadata_key="window",
  original_text_metadata_key="original_text"
)

In [ ]:
nodes=node_parser.get_nodes_from_documents(relevant_sections_llama)

In [ ]:
print(f"length of nodes: {len(nodes)}")
for i in range(len(nodes)):
  print(nodes[i])

length of nodes: 111
Node ID: 5283928e-0292-4290-8821-975c0e1af39a
Text: Production ( edit )     In March 2015 , it was confirmed that
T.S.
Node ID: f71d1792-a34a-4969-a82d-80edacfd41db
Text: Nowlin , who co-wrote the first and wrote the second film ,
would adapt Maze Runner : The Death Cure .
Node ID: d6ea920d-03cb-44bf-911d-fc75bae7a42c
Text: On September 16 , 2015 , it was confirmed that Ball would return
to direct the final film .
Node ID: fe475cc4-61da-4f8b-9e7d-203cca26b250
Text: Maze Runner : the Death Cure   Jump to : navigation , search For
the book the film is based on , see The Death Cure .
Node ID: 53637492-37c1-4405-a6e4-ebdf0fcf30a4
Text: Maze Runner : The Death Cure          Theatrical release poster
Directed by     Wes Ball         Produced by         Ellen Goldsmith -
Vein     Wyck Godfrey     Marty Bowen     Joe Hartwick , Jr .
Node ID: 22e03c34-c916-4465-b4a7-563374a55799
Text: Wes Ball     Lee Stollman             Screenplay by     T.S.
Node ID: b24f9e35-85c5-4673-b

Embed the sentences with their surrounding context and load them into the vector store

In [ ]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
# set global or temporary settings
Settings.llm = OpenAI(model="gpt-4o", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = node_parser

In [ ]:
# build index and the settings will be used automatically
sentence_index = VectorStoreIndex.from_documents(relevant_sections_llama)

In [ ]:
sentence_index.storage_context.persist(persist_dir="/content/drive/MyDrive/ECE1508/sentence_index")

Construct query engine

Component 1: Metadata Replacement Postprocessor  
responsible for replacing the node text with the surrounding context stored in the metadata.

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
postproc= MetadataReplacementPostProcessor(
  target_metadata_key="window"
)

Component 2: Sentence Transformer Re-ranker  
re-ordering the retrieved nodes based on their relevance to the query

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
rerank=SentenceTransformerRerank(
  top_n=2, model="BAAI/bge-reranker-base"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Combining the 2 components into the query engine

In [ ]:
sentence_window_engine=sentence_index.as_query_engine(
  similarity_top_k=5,node_postprocessors=[postproc,rerank]
)

## Generate Answers

In [ ]:
questions=documents[0].metadata.get("question_texts")
for i in range(len(questions)):
  window_response=sentence_window_engine.query(questions[i])
  print(questions[i])
  print(window_response)

when did the new maze runner movie come out
The new Maze Runner movie, "Maze Runner: The Death Cure," was released on January 26, 2018.
when is the death cure coming out in ireland
The release date for "Maze Runner: The Death Cure" in Ireland is not specified in the provided information.
when is the death cure released in the uk
The film "Maze Runner: The Death Cure" was released in the UK on January 26, 2018.
